# Import necessary imports

In [7]:
import torch
import cv2
import easyocr
from ultralytics import YOLO
import time

# License Plate Detection only 

In [9]:
# Load the YOLO model

# model = YOLO("best_model_yolo11s.onnx")
model = YOLO("best_model_yolo11s.pt")

# Open the video file
video_path = "demo.mp4"
cap = cv2.VideoCapture(video_path)

# Set the device for inference
use_gpu = True  # Set to True to use GPU, False for CPU

if use_gpu:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
else:
    device = 'cpu'


frame_count = 0
total_time = 0.0

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        start_time = time.time()
        
        # Run YOLO inference on the frame
        results = model(frame, device=device, verbose=False)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        end_time = time.time()
        elapsed = end_time - start_time
        total_time += elapsed
        frame_count += 1

        fps = 1.0 / elapsed
        cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # Display the annotated frame
        cv2.imshow("YOLO Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

avg_fps = frame_count / total_time if total_time > 0 else 0
print(f"\n[INFO] Processed {frame_count} frames")
print(f"[INFO] Total Time: {total_time:.2f}s")
print(f"[INFO] Average FPS ({'GPU' if use_gpu else 'CPU'}): {avg_fps:.2f}")


[INFO] Processed 631 frames
[INFO] Total Time: 56.46s
[INFO] Average FPS (GPU): 11.18


# License Plate Detection & Recognition

In [29]:
# Use GPU if available
use_gpu = False

## Initialize OCR reader once (Default language is English)
# reader = easyocr.Reader(['en'], gpu=use_gpu)

# Custom recognition model path
reader = easyocr.Reader(['en'], gpu=use_gpu, recog_network='num_plate_ocr')
PREDICTION_ALLOW_LIST = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ.-'

def get_ocr(im, box):
    x1, y1, x2, y2 = map(int, box)
    cropped = im[y1:y2, x1:x2]
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    results = reader.readtext(gray, allowlist=PREDICTION_ALLOW_LIST)
    for result in results:
        text, score = result[1], result[2]
        if score > 0.2:
            return text
    return ""

def draw_boxes_with_ocr(image, boxes, confidences, classes, names):
    for box, conf, cls in zip(boxes, confidences, classes):
        x1, y1, x2, y2 = map(int, box)
        label = f"{names[int(cls)]} {conf:.2f}"
        ocr_text = get_ocr(image, (x1, y1, x2, y2))
        label = ocr_text if ocr_text else label

        color = (0, 255, 0)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return image

def run_video_prediction(model_path="best_model_yolo11s.pt", source=0, use_gpu=True):
    device = 'cuda' if use_gpu else 'cpu'
    model = YOLO(model_path)
    cap = cv2.VideoCapture(source)

    frame_count = 0
    total_time = 0.0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()

        results = model.predict(frame, device=device, verbose=False)[0]
        names = model.names

        boxes = results.boxes.xyxy.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()

        frame = draw_boxes_with_ocr(frame, boxes, confs, classes, names)

        end_time = time.time()
        elapsed = end_time - start_time
        total_time += elapsed
        frame_count += 1

        fps = 1.0 / elapsed
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        cv2.imshow("License Plate Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    avg_fps = frame_count / total_time if total_time > 0 else 0
    print(f"\n[INFO] Processed {frame_count} frames")
    print(f"[INFO] Total Time: {total_time:.2f}s")
    print(f"[INFO] Average FPS ({'GPU' if use_gpu else 'CPU'}): {avg_fps:.2f}")


if __name__ == "__main__":
    run_video_prediction("best_model_yolo11s.pt", source="demo.mp4", use_gpu=use_gpu)


Using CPU. Note: This module is much faster with a GPU.


No Transformation module specified

[INFO] Processed 631 frames
[INFO] Total Time: 552.93s
[INFO] Average FPS (CPU): 1.14


# Custom Recognition Model Placing in ~/.EasyOCR

In [4]:
import shutil
import os

custom_model_name = "num_plate_ocr"

yaml_src = r"G:\My Documents\Digital Neuron Labtech\Project Src\Recognition Model\num_plate_ocr.yaml"
py_src = r"G:\My Documents\Digital Neuron Labtech\Project Src\Recognition Model\num_plate_ocr.py"
pth_src = r"G:\My Documents\Digital Neuron Labtech\Project Src\Recognition Model\num_plate_ocr.pth"

# Destinations (EasyOCR looks here when loading custom models)
dst_yaml = os.path.expanduser(f"~/.EasyOCR/user_network/{custom_model_name}.yaml")
dst_py = os.path.expanduser(f"~/.EasyOCR/user_network/{custom_model_name}.py")
dst_pth = os.path.expanduser(f"~/.EasyOCR/model/{custom_model_name}.pth")

# Copy
shutil.copy(yaml_src, dst_yaml)
shutil.copy(py_src, dst_py)
shutil.copy(pth_src, dst_pth)

print("✅ Custom model files copied to EasyOCR directories.")


✅ Custom model files copied to EasyOCR directories.
